In [162]:
# TODO
# 1. align team names in other leagues
# 2. fix for rounds not fully played
# 3. include international competitions results in elo calc
# 4. graph odds to win over each round
# 5. try to obtain elo from odds
# 6. get/scrape outrights odds [check the odds api]


# european leagues with seasons not ending around May: (Finland, Ireland), Norway, Sweden
# american leagues -||-: Brazil, Canada, Ecuador, USA, Uruguay
# asian leagues -||-: Japan, Korea et al.

In [163]:
import copy
from datetime import datetime
import json
import math
import os
from pathlib import Path
import random

from dotenv import load_dotenv
import pandas as pd
import requests
from tqdm import tqdm

In [164]:
load_dotenv()

API_TOKEN = os.getenv('X-RapidAPI-Key')
HFA = 50
K_FACTOR = 40

In [165]:
def download_elo_data(date=None) -> None:
    if date is None:
        date = datetime.today().strftime('%Y-%m-%d')
    df = pd.read_csv(f"http://api.clubelo.com/{date}")
    Path("data/elo").mkdir(parents=True, exist_ok=True)
    df = df[['Rank', 'Club', 'Country', 'Level', 'Elo']]
    date = date.replace('-', '')
    df.to_csv(f"data/elo/{date}.csv", index=False)

# download_elo_data('2023-07-20')

In [166]:
def api_get_leagues() -> None:
    url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

    params = {"current": "true"}

    headers = {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    print(response.status_code)

    if response.json()['paging']['total'] != 1:
        raise Exception("Error: multiple pages of leagues")

    Path("data/api").mkdir(parents=True, exist_ok=True)
    with open("data/api/leagues.json", "w") as f:
        json.dump(response.json(), f)

# api_get_leagues()

In [167]:
def api_get_fixtures_for_league(league_id: str, season: str) -> None:
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
    params = {"league": league_id, "season": season}

    headers = {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.json()['results'] == 0:
        raise Exception("No results found.")

    if response.json()['paging']['total'] != 1:
        raise Exception("Error: multiple pages of leagues")

    Path("data/api").mkdir(parents=True, exist_ok=True)
    with open(f"data/api/fixtures_{league_id}_{season}.json", "w") as f:
        json.dump(response.json(), f)

# api_get_fixtures_for_league("103", "2024")

In [168]:
def api_get_standings_for_league() -> None:
    pass

In [169]:
def find_league_id(country_code: str, league_name: str) -> str:
    with open("data/api/leagues.json", "r") as f:
        leagues = json.load(f)
        for league in leagues['response']:
            if league['country']['code'] == country_code and league['league']['type'] == 'League' and league['league']['name'] == league_name:
                return league['league']['id']

assert find_league_id("NO", "Eliteserien") == 103
assert find_league_id("PL", "Ekstraklasa") == 106
assert find_league_id("SE", "Allsvenskan") == 113

In [170]:
def find_latest_elo_file() -> str:
    elo_files = os.listdir("data/elo")
    return f'data/elo/{sorted(elo_files)[-1]}'

In [171]:
def get_team_names_from_elo(elo_country_code: str) -> None:
    df = pd.read_csv(find_latest_elo_file())
    df = df[(df['Country'] == elo_country_code) & (df['Level'] == 1)]
    names = sorted(df['Club'].tolist())
    df = pd.DataFrame(names)
    df.to_excel('tmp_team_names_elo.xlsx', index=False)

# get_team_names_from_elo('POL')

In [192]:
def get_team_names_from_api_dump(path: str) -> None:
    with open(path, "r") as f:
        fixtures = json.load(f)
    names = sorted(set([fixture['teams']['home']['name'] for fixture in fixtures['response']]))
    df = pd.DataFrame(names)
    df.to_excel('tmp_team_names_api.xlsx', index=False)

# get_team_names_from_api_dump('data/api/fixtures_106_2023.json')

In [193]:
def get_api_teams_and_elo(date: str, country_code: str) -> pd.DataFrame:
    date = date.replace('-', '')
    elo_data = pd.read_csv(f"data/elo/{date}.csv")

    elo_data = elo_data[(elo_data['Country'] == country_code) & (elo_data['Level'] == 1)]

    team_map_df = pd.read_excel('team_names.xlsx')
    team_map = dict()
    for i, row in team_map_df.iterrows():
        team_map[row['ELO_name']] = row['API_name']

    elo_data['Club'] = elo_data['Club'].apply(lambda x: team_map[x])

    return elo_data[['Club', 'Elo']]

# elo_df = get_api_teams_and_elo('2023-09-20', 'POL')
# elo_df.head()

In [220]:
def build_historical_standings_table_after_n_rounds(league_id: str, season: str, country_code_elo: str, country_code_api: str, after_round: int, elo_date: str) -> tuple[pd.DataFrame, pd.DataFrame]:

    elo_df = get_api_teams_and_elo(elo_date, country_code_elo)

    with open(f"data/api/fixtures_{league_id}_{season}.json", "r") as f:
        fixtures = json.load(f)['response']

    elo_dict = {row['Club']: row['Elo'] for i, row in elo_df.iterrows()}
    points_dict = {row['Club']: 0 for i, row in elo_df.iterrows()}

    for fixture in fixtures:
        round_str = int(fixture['league']['round'].split(' ')[-1])
        if round_str > after_round:
            continue

        home_team = fixture['teams']['home']['name']
        away_team = fixture['teams']['away']['name']

        home_goals = fixture['goals']['home']
        away_goals = fixture['goals']['away']

        home_elo = elo_dict[home_team]
        away_elo = elo_dict[away_team]

        elo_difference = home_elo - away_elo + HFA

        if home_goals > away_goals:
            points_dict[home_team] += 3

            elo_delta = (1 - 1 / (1 + math.pow(10, -elo_difference/400))) * K_FACTOR
        elif home_goals < away_goals:
            points_dict[away_team] += 3

            elo_delta = (0 - 1 / (1 + math.pow(10, -elo_difference/400))) * K_FACTOR
        else:
            points_dict[home_team] += 1
            points_dict[away_team] += 1

            elo_delta = (1 / 2 - 1 / (1 + math.pow(10, -elo_difference/400))) * K_FACTOR

        elo_dict[home_team] += elo_delta
        elo_dict[away_team] -= elo_delta

    points_df = pd.DataFrame(points_dict.items(), columns=['Club', 'Points'])
    elo_df = pd.DataFrame(elo_dict.items(), columns=['Club', 'Elo'])
    standings_df = pd.merge(elo_df, points_df, on='Club', how='inner')

    standings_df = standings_df.sort_values(by=['Points'], ascending=False).reset_index(drop=True)
    standings_df.index += 1
    
    return standings_df

# elo_df, standings_df = build_historical_elo_and_standings_table_after_n_rounds('106', '2023', 'POL', 'PL', 8, '2023-09-20')
# print(elo_df.head())
# print(standings_df.head())

In [221]:
def simulate_season(league_id: str, season: str, after_round: int, standings_df: pd.DataFrame) -> pd.DataFrame:
    with open(f"data/api/fixtures_{league_id}_{season}.json", "r") as f:
        fixtures = json.load(f)['response']

    elo_dict = {row['Club']: row['Elo'] for i, row in standings_df.iterrows()}
    points_dict = {row['Club']: row['Points'] for i, row in standings_df.iterrows()}

    for fixture in fixtures:
        round_str = int(fixture['league']['round'].split(' ')[-1])
        if round_str <= after_round:
            continue

        home_team = fixture['teams']['home']['name']
        away_team = fixture['teams']['away']['name']

        home_elo = elo_dict[home_team]
        away_elo = elo_dict[away_team]

        elo_difference = home_elo - away_elo + HFA

        draw_prob = max(0, 0.3 - abs(elo_difference) / 2000)
        home_win_prob = max(0, 1 / (1 + math.pow(10, -elo_difference/400)) - draw_prob / 2)
        away_win_prob = max(0, 1 - home_win_prob - draw_prob)

        result = random.choices(['home_win', 'away_win', 'draw'], [home_win_prob, away_win_prob, draw_prob])[0]

        if result == 'home_win':
            points_dict[home_team] += 3
        elif result == 'away_win':
            points_dict[away_team] += 3
        elif result == 'draw':
            points_dict[home_team] += 1
            points_dict[away_team] += 1
        
    points_df = pd.DataFrame(points_dict.items(), columns=['Club', 'Points'])
    elo_df = pd.DataFrame(elo_dict.items(), columns=['Club', 'Elo'])
    season_standings_df = pd.merge(points_df, elo_df, on='Club', how='inner')

    season_standings_df = season_standings_df.sort_values(by=['Points'], ascending=False)

    return season_standings_df

# elo_df = get_api_teams_and_elo('2023-09-20', 'POL')
# standings_df = build_historical_standings_table_after_n_rounds('106', '2023', 'POL', 'PL', 8, '2023-09-20')
# season_df = simulate_season('106', '2023', 8, elo_df, standings_df)
# season_df.head()

In [223]:
def run_multiple_sims(league_id: str, season: str, elo_country_code: str, api_country_code: str, after_round: int, elo_date: str, number_of_sims: int, number_of_winning_places: int) -> pd.DataFrame:
    standings_df = build_historical_standings_table_after_n_rounds(league_id, season, elo_country_code, api_country_code, after_round, elo_date)
    print(standings_df)

    winners = dict()
    number_of_successful_sims = 0

    for _ in tqdm(range(number_of_sims)):
        winners_df = simulate_season(league_id, season, after_round, standings_df)
        if winners_df.iloc[number_of_winning_places - 1]['Points'] == winners_df.iloc[number_of_winning_places]['Points']:
            continue
        number_of_successful_sims += 1
        for i in range(number_of_winning_places):
            try:
                winners[winners_df.iloc[i]['Club']] += 1
            except KeyError:
                winners[winners_df.iloc[i]['Club']] = 1

    df = pd.DataFrame(list(winners.items()), columns=['Club', 'Wins'])
    df['% winrate'] = round(df['Wins'] / number_of_successful_sims * 100)
    df['Expected odds'] = round(number_of_successful_sims / df['Wins'], 2)
    df = df.sort_values(by=['Wins'], ascending=False).reset_index(drop=True)
    df.index += 1
    print(f'{number_of_successful_sims} simulations')
    print(f'{number_of_winning_places} winning places')
    return df

league_id = '106'
season = '2023'
elo_country_code = 'POL'
api_country_code = 'PL'
after_round = 12
elo_date = '2023-07-20'
number_of_sims = 1000
number_of_winning_places = 3
run_multiple_sims(league_id, season, elo_country_code, api_country_code, after_round, elo_date, number_of_sims, number_of_winning_places)

                   Club          Elo  Points
1           Jagiellonia  1461.948026      26
2         Slask Wroclaw  1451.543222      26
3     Raków Częstochowa  1536.927966      25
4           Lech Poznan  1541.601597      24
5        Zaglebie Lubin  1387.647356      20
6        Legia Warszawa  1445.543007      20
7        Pogon Szczecin  1435.832380      19
8       Cracovia Krakow  1366.679714      17
9           Widzew Łódź  1317.967678      17
10       Radomiak Radom  1344.512475      15
11        Piast Gliwice  1430.919713      14
12        Gornik Zabrze  1375.311464      13
13         Warta Poznań  1346.138186      13
14          Stal Mielec  1281.394744      12
15        Korona Kielce  1307.198095      11
16  Puszcza Niepołomice  1260.545662      10
17         Ruch Chorzów  1231.767491       7
18             ŁKS Łódź  1216.223373       7


100%|██████████| 1000/1000 [00:06<00:00, 143.33it/s]

896 simulations
3 winning places


,Club,Wins,% winrate,Expected odds
1,Lech Poznan,776,87.0,1.15
2,Raków Częstochowa,771,86.0,1.16
3,Jagiellonia,494,55.0,1.81
4,Slask Wroclaw,456,51.0,1.96
5,Legia Warszawa,91,10.0,9.85
6,Pogon Szczecin,58,6.0,15.45
7,Zaglebie Lubin,22,2.0,40.73
8,Piast Gliwice,12,1.0,74.67
9,Cracovia Krakow,5,1.0,179.20
10,Gornik Zabrze,2,0.0,448.00


In [197]:
# elo_difference = -200
# elo_delta_w = (1 - 1 / (1 + math.pow(10, -elo_difference/400))) * 40
# elo_delta_d = (1 / 2 - 1 / (1 + math.pow(10, -elo_difference/400))) * 40
# elo_delta_l = (0 - 1 / (1 + math.pow(10, -elo_difference/400))) * 40
# 2 * elo_delta_w + 4 * elo_delta_d + 6 * elo_delta_l